In [ ]:
!pip install langchain langgraph langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.8 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
import os, getpass

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [ ]:
def manager_node(state):
    task_input = state.get("task", "")
    input = state.get("input","")
    prompt = f"""
    You are a task router. Based on the user request below, decide whether it is a:
    - translate
    - summarize
    - calculate

    Respond with only one word (translate, summarize, or calculate).

    Task: {task_input}
    """
    decision = llm.invoke(prompt).content.strip().lower()
    return {"agent": decision, "input":input}

In [ ]:
# Worker Nodes
def translator_node(state):
    text = state.get("input", "")
    prompt = f"Act like You are a translator. Only respond with the English translation of the text below:\n\n{text}"
    result = llm.invoke(prompt).content
    return {"result": result}

def summarizer_node(state):
    text = state.get("input", "")
    prompt = f"Summarize the following in 1-2 lines:\n\n{text}"
    result = llm.invoke(prompt).content
    return {"result": result}

def calculator_node(state):
    expression = state.get("input", "")
    prompt = f"Please calculate and return the result of:\n{expression}"
    result = llm.invoke(prompt).content
    return {"result": result}

In [ ]:
def route_by_agent(state):
    return {
        "translate": "Translator",
        "summarize": "Summarizer",
        "calculate": "Calculator",
        "input": state.get("input", "")
    }.get(state.get("agent", ""), "Default")


In [ ]:
def default_node(state):
    return {"result": "Sorry, I couldn't understand the task."}

In [ ]:
from langgraph.graph import StateGraph
g = StateGraph(dict)
g.add_node("Manager", manager_node)
g.add_node("Translator", translator_node)
g.add_node("Summarizer", summarizer_node)
g.add_node("Calculator", calculator_node)
g.add_node("Default", default_node)
g.set_entry_point("Manager")
g.add_conditional_edges("Manager", route_by_agent)
g.set_finish_point("Translator")
g.set_finish_point("Summarizer")
g.set_finish_point("Calculator")
g.set_finish_point("Default")
graph = g.compile()

In [ ]:
# Translate Example
print(graph.invoke({
    "task": "Can you translate this?",
    "input": "Bonjour le monde"
}))


{'result': 'Hello world'}


In [ ]:
# Summarize Example
print(graph.invoke({
    "task": "Please summarize the following",
    "input": "LangGraph helps you build flexible multi-agent workflows in Python..."
}))


{'result': 'LangGraph enables the creation of adaptable multi-agent workflows in Python, streamlining complex task management and collaboration.'}


In [ ]:

respcal = graph.invoke({
    "task": "What is 12 * 8 + 5?",
    "input": "12 * 8 + 5"
})

In [ ]:
print(respcal['result'])

The result of the calculation \(12 \times 8 + 5\) is:

\[
12 \times 8 = 96
\]

Then, adding 5:

\[
96 + 5 = 101
\]

So, the final result is \(101\).


In [ ]:
# Unknown Example
print(graph.invoke({
    "task": "Can you dance?",
    "input": "foo"
}))

{'result': 'It seems like there may have been an error or misunderstanding, as "foo" does not provide enough context or content to summarize. Please provide more information or clarify what you would like summarized!'}
